<a href="https://colab.research.google.com/github/VuBrian22/Torrent-to-Google-Drive-/blob/main/Torrent2GoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Torrent to Google Drive (Updated with New Features) - Brian Vu**
**Credit: r12habh (https://github.com/r12habh)**

# **STEP 1 : MOUNT YOUR GOOGLE DRIVE WHERE YOU WANT TO STORE THE DATA**

In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


# **STEP 2 : INSTALL THE PYTHON LIBRARIES**

In [2]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>



!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

from IPython.display import HTML, clear_output

clear_output()

print("Server Started Successfully")



Server Started Successfully


# *Following steps for Google Drive Upload (Personal Drive)*
<br><center><img src='https://drive.google.com/uc?id=1iqeDl-Jdv6FyeswW4Jt7bDN-e6VdHMb2' height="100" /></center> 


# **STEP 3 : UPLOAD .TORRENT FILE OR PASTE MAGNET LINK [for GDrive]**
**This supports simultaneous torrent uploads :D**

In [5]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Upload torrent file</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5>
#@markdown <h5>No parallel downloading using .torrent files, use magnet for that</h5><br>


from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Saving Now You See Me 2 (2016) [1080p] [BluRay] [YTS.MX].torrent to Now You See Me 2 (2016) [1080p] [BluRay] [YTS.MX].torrent


In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>


params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


# *Following steps for Team Drive Upload (Shared Drive)*
<br><center><img src='https://drive.google.com/uc?id=1AC4hpO-pE2FyHzTQBrwq27kGbZDOeVuO' height="100" /></center> 


# **STEP 3 : UPLOAD .TORRENT FILE OR PASTE MAGNET LINK [for Team Drive]**

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Upload torrent file</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5>
#@markdown <h5>Doesn't support parallel downloding, use magnet method for that</h5><br>

#@markdown <h3>Enter the path of TD as shown in the video :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]


from google.colab import files

source = files.upload()
params = {
    "save_path": path,
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

clear_output()

print("Started Successfully")


In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>

#@markdown <h3>Enter the path of TD as shown in the video :</h3>
path = "/content/drive/Shared drives/Google_Drive/Torrent" #@param {type:"string"}
check=[]
params = {"save_path": path}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

clear_output()

print("Started Successfully")   


# **STEP 4 : START CLOUD UPLOAD**

In [6]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Now You See Me 2 (2016) [1080p] [YTS.AG] complete
